In [242]:
import pandas as pd
import numpy as np
import os
import torch
from torch.utils.data import Dataset, DataLoader
import re

In [13]:
proj_path = "/Users/gopal/Google Drive/_Research/Research projects/ML/manclassify/app_data/Thailand"
class_path = os.path.join(proj_path,"Thailand_classification")
ts_path = os.path.join(proj_path,"Thailand_download_timeseries")
# pd.read_csv("
os.listdir(class_path)

loc_id = 0

s2_csv_name = f"pt_ts_loc{loc_id}_s2.csv"
s2_csv_name



'pt_ts_loc0_s2.csv'

In [23]:
os.listdir(class_path)

['countif.xlsx', 'location_classification.csv']

In [253]:
class s2Dataset(Dataset):
    """Sentinel 2 dataset"""
    
    def __init__(self, proj_path, class_colname):
        """
        Args:
            proj_path (string): path to manclassify project
        """
        self.proj_path = proj_path
        proj_normpath = os.path.normpath(proj_path)
        proj_dirname = proj_normpath.split(os.sep)[-1]
        self.proj_name = re.sub("_classification$","",proj_dirname)
        self.class_path = os.path.join(proj_path, self.proj_name + "_classification")
        self.ts_path = os.path.join(proj_path, self.proj_name + "_download_timeseries")
        self.pt_classes = pd.read_csv(os.path.join(self.class_path,"location_classification.csv"))
        self.pt_classes = classes[['loc_id', class_colname]].dropna()
        # self.pt_classes['loc_id'] = self.pt_classes['loc_id'] + 10.5 # for testing index only
        self.classes = pd.unique(self.pt_classes[class_colname])
        self.labels = self.pt_classes.assign(val = 1).pivot_table(columns = class_colname, index = 'loc_id', values = 'val', fill_value= 0)

    
    def __getitem__(self, idx):
        date_range = pd.to_datetime(['2019-06-01','2020-05-31'])
        loc_id = self.labels.index[idx]
        s2_csv_name = f"pt_ts_loc{loc_id}_s2.csv"
        s2_csv_path = os.path.join(self.ts_path, s2_csv_name)
        s2_ts = pd.read_csv(s2_csv_path)
        
        # extract dates from image ids
        s2_ts['datestr'] = [re.sub("(^[0-9]+)[a-zA-Z].*","\\1",x) for x in s2_ts.image_id]
        s2_ts['date'] = pd.to_datetime(s2_ts.datestr, format = "%Y%m%d")
        
        # subset to cloud-free days AND within date_range
        s2_ts = s2_ts[(s2_ts.date >= date_range[0]) & (s2_ts.date <= date_range[1]) & (s2_ts.cloudmask == 0)]
        
        # calculate day from startday
        date_diff = (s2_ts.date - date_range[0])
        s2_ts['day'] = [x.days for x in date_diff]
        
        
        s2_ts_x = s2_ts[['B8','B4','B3','B2','day']]
        s2_tensor = torch.tensor(s2_ts_x.to_numpy())
        
        return s2_tensor
        
    def __len__(self):
        return self.pt_classes.shape[0]

In [254]:
date_rangeX = pd.to_datetime(['2019-06-01','2020-05-31'])
s2_dataset = s2Dataset(proj_path = proj_path, class_colname = 'Subclass2019')
classes = pd.read_csv(os.path.join(s2_dataset.class_path,"location_classification.csv"))
# c2 = classes[['loc_id', 'Subclass2019']]#[!classes['Subclass2019'].isnan]
df = s2_dataset.__getitem__(4)
df

tensor([[3904,  235,  471,  236,   20],
        [2089,  265,  451,  383,   40],
        [3745,  280,  538,  329,   80],
        [3791,  225,  474,  269,   80],
        [4465,  392,  712,  562,  120],
        [3285,  215,  439,  231,  125],
        [3276,  179,  404,  187,  125],
        [3411,  217,  441,  229,  145],
        [3380,  155,  358,  141,  145],
        [2067,  223,  416,  384,  150],
        [2161,  238,  434,  423,  150],
        [3332,  279,  469,  284,  160],
        [3310,  335,  578,  408,  165],
        [3250,  296,  529,  371,  165],
        [3296,  519,  776,  662,  205],
        [2720,  396,  617,  530,  205],
        [3341,  555,  800,  641,  230],
        [3272,  605,  857,  729,  230],
        [2840,  167,  353,  156,  240],
        [2720,  158,  339,  169,  240],
        [2194,  402,  575,  485,  270],
        [2137,  311,  482,  406,  270],
        [3316,  581,  768,  579,  285],
        [3304,  522,  718,  522,  285],
        [4199,  302,  789,  311,  310],


In [132]:
df['val'] = 1
labels_one_hot = df.pivot_table(columns = 'Subclass2019', index = 'loc_id', values = 'val', fill_value= 0)
labels_one_hot

Subclass2019,Crop(Double),Crop(Single),Forest,Golf,Mixed,Plantation,Unsure,Urban,Water
loc_id,,,,,,,,,
0,0,0,0,0,0,1,0,0,0
1,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
496,0,1,0,0,0,0,0,0,0
497,0,1,0,0,0,0,0,0,0
498,0,0,0,0,0,1,0,0,0


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [29]:
proj_path

'/Users/gopal/Google Drive/_Research/Research projects/ML/manclassify/app_data/Thailand'